In [15]:
from typing import Tuple
from pathlib import Path
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.ensemble import RandomForestClassifier
import os



from fairlearn.datasets import fetch_adult
from fairlearn.metrics import (
    demographic_parity_difference,
    demographic_parity_ratio,
    equalized_odds_difference,
    equalized_odds_ratio,
)
from fairlearn.reductions import ExponentiatedGradient, DemographicParity

# -------------------------
# Configuration
# -------------------------
np.random.seed(42)
sns.set_style("whitegrid")
plt.rcParams["figure.figsize"] = (15, 6)

OUTPUT_DIR = Path("second")
OUTPUT_DIR.mkdir(exist_ok=True)

In [16]:
def load_and_preprocess_data(test_size: float = 0.3, random_state: int = 42):
    """
    Loads the Adult dataset locally and returns scaled train/test splits
    plus the protected attribute 'sex'.
    
    Returns:
        X_train_df, X_test_df, y_train, y_test, sex_train, sex_test
    """

    # Column names from UCI Adult dataset definition
    columns = [
        "age", "workclass", "fnlwgt", "education", "education-num",
        "marital-status", "occupation", "relationship", "race", "sex",
        "capital-gain", "capital-loss", "hours-per-week", "native-country", "income"
    ]

    # Load from your local path instead of fetch_adult()
    df = pd.read_csv("adult/adult.data", names=columns, sep=",", skipinitialspace=True)

    # Keep protected attribute
    sex = df["sex"].copy()

    # Convert target to 0/1
    y = df["income"].map({">50K": 1, "<=50K": 0})

    # Drop target from input features
    X = df.drop(columns=["income","fnlwgt"])

    # One-hot encode categorical features
    X_encoded = pd.get_dummies(X, drop_first=True)

    # Train/validation split (stratified to keep income balance)
    X_train, X_test, y_train, y_test, sex_train, sex_test = train_test_split(
        X_encoded, y, sex, test_size=test_size,
        random_state=random_state, stratify=y
    )

    # Scale all features (safe even after one-hot encoding)
    scaler = StandardScaler()
    X_train_scaled = scaler.fit_transform(X_train)
    X_test_scaled = scaler.transform(X_test)

    # Convert back to DataFrame
    X_train_df = pd.DataFrame(X_train_scaled, columns=X_encoded.columns, index=X_train.index)
    X_test_df = pd.DataFrame(X_test_scaled, columns=X_encoded.columns, index=X_test.index)

    return X_train_df, X_test_df, y_train, y_test, sex_train, sex_test


In [17]:
def measure_all_metrics(y_test, y_pred_baseline, y_pred_fair, sex_test):
    """
    Computes fairness metrics for Baseline vs Fair Model.
    Returns a DataFrame summary.
    """

    results = []

    for name, preds in [
        ("Baseline (Unfair)", y_pred_baseline),
        ("Fair (ExponentiatedGradient)", y_pred_fair),
    ]:
        preds = pd.Series(preds, index=y_test.index)

        dpd = demographic_parity_difference(y_test, preds, sensitive_features=sex_test)
        dpr = demographic_parity_ratio(y_test, preds, sensitive_features=sex_test)
        eod = equalized_odds_difference(y_test, preds, sensitive_features=sex_test)
        eor = equalized_odds_ratio(y_test, preds, sensitive_features=sex_test)

        male_rate = preds[sex_test == "Male"].mean()
        female_rate = preds[sex_test == "Female"].mean()
        gap = abs(male_rate - female_rate)

        print(f"\n{name}")
        print(f"  DPD: {dpd:.4f}   (→ 0 = parity)")
        print(f"  DPR: {dpr:.4f}   (→ 1 = perfect)")
        print(f"  EOD: {eod:.4f}")
        print(f"  EOR: {eor:.4f}")
        print(f"  Male >$50K:   {male_rate:.3f}")
        print(f"  Female >$50K: {female_rate:.3f}")
        print(f"  Prediction Gap: {gap:.3f}")

        results.append({
            "Model": name,
            "DPD": dpd,
            "DPR": dpr,
            "EOD": eod,
            "EOR": eor,
            "Male_Positive_Rate": male_rate,
            "Female_Positive_Rate": female_rate,
            "Gap": gap
        })

    return pd.DataFrame(results)

In [18]:
def create_comparison_visualizations_individual(baseline_acc, fair_acc, fairness_df, model_name, output_dir="second"):
    """
    Create and save individual visualizations for baseline vs fair model.
    Each metric is saved as a separate file with the model name in the filename,
    including a summary panel showing fairness vs accuracy trade-off.
    """
    os.makedirs(output_dir, exist_ok=True)
    models = fairness_df["Model"].values

    # ---------------- 1) Accuracy ----------------
    plt.figure(figsize=(8,6))
    plt.bar(models, [baseline_acc, fair_acc], color=["#e74c3c", "#2ecc71"])
    plt.title(f"{model_name}: Model Accuracy", fontsize=14, fontweight="bold")
    plt.ylim([0.0, 1.0])
    for i, acc in enumerate([baseline_acc, fair_acc]):
        plt.text(i, acc + 0.01, f"{acc:.3f}", ha="center", fontsize=10)
    plt.tight_layout()
    plt.savefig(os.path.join(output_dir, f"accuracy_{model_name}.png"))
    plt.close()

    # ---------------- 2) Demographic Parity Difference (DPD) ----------------
    plt.figure(figsize=(8,6))
    dpd_values = fairness_df["DPD"].values
    plt.bar(models, dpd_values, color=["#e74c3c", "#2ecc71"])
    plt.axhline(0.0, linestyle="--", color="black")
    plt.title(f"{model_name}: Demographic Parity Difference (DPD)", fontsize=14, fontweight="bold")
    for i, val in enumerate(dpd_values):
        offset = 0.01 if val >= 0 else -0.03
        plt.text(i, val + offset, f"{val:.3f}", ha="center", fontsize=10)
    plt.tight_layout()
    plt.savefig(os.path.join(output_dir, f"dpd_{model_name}.png"))
    plt.close()

    # ---------------- 3) Disparate Impact Ratio (DPR) ----------------
    plt.figure(figsize=(8,6))
    dpr_values = fairness_df["DPR"].values
    plt.bar(models, dpr_values, color=["#e74c3c", "#2ecc71"])
    plt.axhline(1.0, linestyle="--", color="black", label="Perfect = 1.0")
    plt.title(f"{model_name}: Disparate Impact Ratio (DPR)", fontsize=14, fontweight="bold")
    for i, val in enumerate(dpr_values):
        plt.text(i, val + 0.02, f"{val:.3f}", ha="center", fontsize=10)
    plt.legend()
    plt.tight_layout()
    plt.savefig(os.path.join(output_dir, f"dpr_{model_name}.png"))
    plt.close()

    # ---------------- 4) Equalized Odds Difference (EOD) ----------------
    plt.figure(figsize=(8,6))
    eod_values = fairness_df["EOD"].values
    gap_values = fairness_df["Gap"].values
    bars = plt.bar(models, eod_values, color=["#e74c3c", "#2ecc71"])
    plt.axhline(0.0, linestyle="--", color="black")
    plt.title(f"{model_name}: Equalized Odds Difference (EOD)", fontsize=14, fontweight="bold")
    legend_labels = [f"{name}\nEOD: {eod:.3f}, Gap: {gap:.3f}" for name, eod, gap in zip(models, eod_values, gap_values)]
    plt.legend(bars, legend_labels)
    plt.tight_layout()
    plt.savefig(os.path.join(output_dir, f"eod_{model_name}.png"))
    plt.close()

    # ---------------- 5) Positive Prediction Rates by Gender ----------------
    plt.figure(figsize=(8,6))
    width = 0.35
    x = range(len(models))
    male_rates = fairness_df["Male_Positive_Rate"].fillna(0).values
    female_rates = fairness_df["Female_Positive_Rate"].fillna(0).values
    plt.bar([p - width/2 for p in x], male_rates, width, color="#3498db", label="Male")
    plt.bar([p + width/2 for p in x], female_rates, width, color="#f39c12", label="Female")
    plt.xticks(x, models, rotation=15, ha="right")
    plt.title(f"{model_name}: Positive Prediction Rates by Gender", fontsize=14, fontweight="bold")
    plt.ylabel("Positive Rate")
    legend_labels_rates = [
        f"{m}\nMale: {mr*100:.1f}%, Female: {fr*100:.1f}%, Gap: {abs(mr-fr)*100:.1f}%"
        for m, mr, fr in zip(models, male_rates, female_rates)
    ]
    plt.legend(legend_labels_rates, loc="upper right", fontsize=10)
    plt.tight_layout()
    plt.savefig(os.path.join(output_dir, f"positive_rates_{model_name}.png"))
    plt.close()

    # ---------------- 6) Summary panel (DPD reduction, DPR improvement, Accuracy loss) ----------------
    dpd_base = abs(dpd_values[0])
    dpd_improve = (1 - abs(dpd_values[1]) / dpd_base) * 100
    denom = 1 - dpr_values[0] if (1 - dpr_values[0]) != 0 else 1e-9
    dpr_improve = ((dpr_values[1] - dpr_values[0]) / denom) * 100
    acc_loss = (baseline_acc - fair_acc) * 100

    summary = pd.DataFrame({
        "Metric": ["DPD Reduction %", "DPR Improvement %", "Accuracy Loss %"],
        "Value":  [dpd_improve, dpr_improve, acc_loss]
    })

    plt.figure(figsize=(8,6))
    plt.barh(summary["Metric"], summary["Value"], color="#3498db")
    plt.title(f"{model_name}: Fairness vs Accuracy Trade-off", fontsize=14, fontweight="bold")
    for i, v in enumerate(summary["Value"]):
        plt.text(v + 1, i, f"{v:.1f}%", va="center", fontsize=10)
    plt.tight_layout()
    plt.savefig(os.path.join(output_dir, f"tradeoff_{model_name}.png"))
    plt.close()

    print(f"✅ All individual plots saved in '{output_dir}' for {model_name}")


In [19]:
def create_summary_table(fairness_df, acc_dict):
    rows = []
    for idx, row in fairness_df.iterrows():
        acc = acc_dict[row["Model"]]
        rows.append({
            "Model": row["Model"],
            "Accuracy": round(acc, 4),
            "DPD": round(row["DPD"], 4),
            "DPR": round(row["DPR"], 4),
            "EOD": round(row["EOD"], 4),
            "EOR": round(row["EOR"], 4),
            "Male Positive %": f"{row['Male_Positive_Rate']*100:.1f}%",
            "Female Positive %": f"{row['Female_Positive_Rate']*100:.1f}%"
        })
    summary_df = pd.DataFrame(rows)
    summary_df.to_csv("fair_model_results.csv", index=False)
    return summary_df

In [20]:
# ==== RUN PIPELINE IN NOTEBOOK ====

# 1) Load + preprocess
X_train, X_test, y_train, y_test, sex_train, sex_test = load_and_preprocess_data()

# 2) Define models to run
models = {
    "LogisticRegression": LogisticRegression(max_iter=1000, random_state=42),
    "RandomForest": RandomForestClassifier(n_estimators=100, random_state=42)
}

results = []

for model_name, estimator in models.items():
    print(f"\n=== {model_name} ===")
    
    # ---- Baseline (unconstrained) ----
    baseline_model = estimator
    baseline_model.fit(X_train, y_train)
    y_pred_baseline = pd.Series(baseline_model.predict(X_test), index=y_test.index)
    baseline_acc = accuracy_score(y_test, y_pred_baseline)
    print(f"Baseline accuracy: {baseline_acc:.4f}")
    
    # ---- Fairness-constrained (Exponentiated Gradient + Demographic Parity) ----
    mitigator = ExponentiatedGradient(
        estimator=estimator,
        constraints=DemographicParity(),
        max_iter=50
    )
    mitigator.fit(X_train, y_train, sensitive_features=sex_train)
    y_pred_fair = pd.Series(mitigator.predict(X_test), index=y_test.index)
    fair_acc = accuracy_score(y_test, y_pred_fair)
    print(f"Fair model accuracy: {fair_acc:.4f}")
    
    # ---- Compute fairness metrics ----
    fairness_df = measure_all_metrics(y_test, y_pred_baseline, y_pred_fair, sex_test)
    display(fairness_df)
    
    # ---- Visualization ----
    create_comparison_visualizations_individual(baseline_acc, fair_acc, fairness_df, model_name, output_dir="second")
    print(f"\n📊 Saved figure for {model_name}: second/fair_model_comparison_{model_name}.png")
    
    # ---- Summary table ----
    acc_dict = {
    "Baseline (Unfair)": baseline_acc,
    "Fair (ExponentiatedGradient)": fair_acc
}
    summary_df = create_summary_table(fairness_df, acc_dict)
    display(summary_df)
    print(f"📂 CSV saved for {model_name}: second/fair_model_results.csv")
    
    # Store for reference if needed
    results.append({
        "model_name": model_name,
        "baseline_acc": baseline_acc,
        "fair_acc": fair_acc,
        "fairness_df": fairness_df,
        "summary_df": summary_df
    })



=== LogisticRegression ===
Baseline accuracy: 0.8532
Fair model accuracy: 0.8358

Baseline (Unfair)
  DPD: 0.1727   (→ 0 = parity)
  DPR: 0.3291   (→ 1 = perfect)
  EOD: 0.0741
  EOR: 0.2494
  Male >$50K:   0.257
  Female >$50K: 0.085
  Prediction Gap: 0.173

Fair (ExponentiatedGradient)
  DPD: 0.0065   (→ 0 = parity)
  DPR: 0.9621   (→ 1 = perfect)
  EOD: 0.3174
  EOR: 0.5107
  Male >$50K:   0.171
  Female >$50K: 0.165
  Prediction Gap: 0.006


,Model,DPD,DPR,EOD,EOR,Male_Positive_Rate,Female_Positive_Rate,Gap
0,Baseline (Unfair),0.172668,0.329137,0.074137,0.249445,0.257382,0.084714,0.172668
1,Fair (ExponentiatedGradient),0.006494,0.962076,0.317444,0.510697,0.171233,0.164739,0.006494


✅ All individual plots saved in 'second' for LogisticRegression

📊 Saved figure for LogisticRegression: second/fair_model_comparison_LogisticRegression.png


,Model,Accuracy,DPD,DPR,EOD,EOR,Male Positive %,Female Positive %
0,Baseline (Unfair),0.8532,0.1727,0.3291,0.0741,0.2494,25.7%,8.5%
1,Fair (ExponentiatedGradient),0.8358,0.0065,0.9621,0.3174,0.5107,17.1%,16.5%


📂 CSV saved for LogisticRegression: second/fair_model_results.csv

=== RandomForest ===
Baseline accuracy: 0.8457
Fair model accuracy: 0.8046

Baseline (Unfair)
  DPD: 0.1856   (→ 0 = parity)
  DPR: 0.3282   (→ 1 = perfect)
  EOD: 0.0875
  EOR: 0.2615
  Male >$50K:   0.276
  Female >$50K: 0.091
  Prediction Gap: 0.186

Fair (ExponentiatedGradient)
  DPD: 0.0256   (→ 0 = parity)
  DPR: 0.9046   (→ 1 = perfect)
  EOD: 0.0800
  EOR: 0.5808
  Male >$50K:   0.268
  Female >$50K: 0.243
  Prediction Gap: 0.026


,Model,DPD,DPR,EOD,EOR,Male_Positive_Rate,Female_Positive_Rate,Gap
0,Baseline (Unfair),0.185602,0.328150,0.087469,0.261516,0.276256,0.090653,0.185602
1,Fair (ExponentiatedGradient),0.025605,0.904635,0.079982,0.580756,0.268493,0.242888,0.025605


✅ All individual plots saved in 'second' for RandomForest

📊 Saved figure for RandomForest: second/fair_model_comparison_RandomForest.png


,Model,Accuracy,DPD,DPR,EOD,EOR,Male Positive %,Female Positive %
0,Baseline (Unfair),0.8457,0.1856,0.3282,0.0875,0.2615,27.6%,9.1%
1,Fair (ExponentiatedGradient),0.8046,0.0256,0.9046,0.0800,0.5808,26.8%,24.3%


📂 CSV saved for RandomForest: second/fair_model_results.csv
